In [1]:
from relativisticpy.core.indices import Indices, Idx
from relativisticpy.gr.metric import Metric, MetricIndices
from relativisticpy.gr.derivative import Derivative
from relativisticpy.gr.connection import Connection

In [9]:
comps = '[[1, 0], [0, r**2]]'
basis = '[r, theta]'

g_ab = Metric.from_string('_{a}_{b}', comps, basis)
g_bc = Metric.from_string('_{b}_{c}', comps, basis)
g_be = Metric.from_string('_{b}_{e}', comps, basis)
g_ec = Metric.from_string('_{e}_{c}', comps, basis)

d_e = Derivative(Indices(Idx('e')))
d_c = Derivative(Indices(Idx('c')))
d_b = Derivative(Indices(Idx('b')))

g_ab.indices

In [10]:
from relativisticpy.providers import SymbolArray, transpose_list
from itertools import product
from operator import itemgetter

a = SymbolArray([[[1,0], [2,2]], [[2,2], [2,2]]])
b = SymbolArray([[2, 2], [2, 2]])

summed_index_locations = transpose_list([[1,1]])
all = [(idxs[:len(a.shape)],idxs[len(a.shape):]) for idxs in list(product(*[range(i) for i in a.shape + b.shape])) if itemgetter(*summed_index_locations[0])(idxs[:len(a.shape)]) == itemgetter(*summed_index_locations[1])(idxs[len(a.shape):])]

c1 = len(summed_index_locations[0]) == len(a.shape)
c2 = len(summed_index_locations[1]) == len(b.shape)

def generator(idx):
    if not c1 and not c2:
        return [(IndicesA, IndicesB) for (IndicesA, IndicesB) in all if itemgetter(*A_indices_not_summed)(IndicesA) == itemgetter(*result_indices_in_A)(idx) and itemgetter(*B_indices_not_summed)(IndicesB) == itemgetter(*result_indices_in_B)(idx)]
    elif c1 and not c2: 
        return [(IndicesA, IndicesB) for (IndicesA, IndicesB) in all if itemgetter(*B_indices_not_summed)(IndicesB) == itemgetter(*result_indices_in_B)(idx)]
    elif c2 and not c1:
        return [(IndicesA, IndicesB) for (IndicesA, IndicesB) in all if itemgetter(*A_indices_not_summed)(IndicesA) == itemgetter(*result_indices_in_A)(idx)]

In [11]:
import re
def is_repr_a(arg: str): return bool(re.search("^((\^|\_)(\{)(\}))+$", re.sub('[^\^^\_^\{^\}]',"", arg).replace(" ",''))) if isinstance(arg, str) else False
def is_running(arg: str): return not bool(re.search('^[^=]*(\:)([0-9]+)[^=]*$', arg))
def get_symbol(arg: str): return re.search(r'[a-zA-Z]+', arg).group() if re.search(r'[a-zA-Z]+', arg) else None
def get_values(arg: str): return re.search(r'[0-9]+', arg).group() if re.search(r'[0-9]+', arg) else None
def covariant(arg: str): return arg[0] == '_' if isinstance(arg, str) else True # Always default to coveriant
def split(arg: str): return [item for item in re.split('(?=[_^])', arg) if item] if is_repr_a(arg) else arg
def descerialize(arg: str): return Indices(*[Idx(symbol = get_symbol(i), covariant=covariant(i), values=int(get_values(i)) if not is_running(i) else None) for i in split(arg)]) if is_repr_a(arg) else None

In [12]:
a = MetricIndices.from_string('_{a}^{c:12432}')
a

In [13]:
(d_b*(-1*g_ec) + d_c*g_be + d_e*g_bc).indices.indices # Fix => if user specifies the resulting indices then we can just map them as specified

(_b  , _e  , _c  )

In [14]:
(d_c*g_be + d_e*g_bc + d_b*(-1*g_ec)).indices.indices # Fix => if user specifies the resulting indices then we can just map them as specified

(_c  , _b  , _e  )

In [15]:
r = d_b*(-1*g_ec)
r.indices.indices

(_b  , _e  , _c  )

In [16]:
res = (g_ab.inv*(d_c*g_be + d_e*g_bc + d_b*(-1*g_ec)))/2
res.components

[[[0, 0], [0, 1/r]], [[0, -r], [1/r, 0]]]

In [17]:
a = Connection(Indices(-Idx('e'), Idx('a'), Idx('c')), metric = g_ab)

In [18]:
a[Indices(-Idx('a'), Idx('e'), Idx('c'))] = res

hello there


In [19]:
a.components

[[[0, 0], [0, -r]], [[0, 1/r], [1/r, 0]]]